In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data.csv')

data.drop_duplicates(inplace=True)
data.sort_values(by=['period'], inplace=True, ascending=False)

data.to_csv('data.csv', index=False)
data.head()

In [ ]:
data.drop("price", axis=1, inplace=True)

data['count_of_the_day'] = data['period'].apply(lambda x: int(str(x)[8:]))

data['period'] = data['period'].apply(lambda x: int(str(x)[0:8]))

data.head()

In [ ]:
data['target_colour'] = data['target_colour'] - 1

data.head()

In [ ]:
def previous_counts(row, df):
    period = row['period']
    count_of_the_day = row['count_of_the_day']
    same_day_records = df[(df['period'] == period) & (df['count_of_the_day'] < count_of_the_day)]
    if same_day_records.empty:
        return pd.Series({'previous_50_numbers': [], 'previous_50_colours': []})
    lower_limit = max(count_of_the_day - 50, same_day_records['count_of_the_day'].min())
    prev_records = same_day_records[same_day_records['count_of_the_day'].between(lower_limit, count_of_the_day - 1)]
    return pd.Series({'previous_50_numbers': prev_records['target_number'].values, 'previous_50_colours': prev_records['target_colour'].values})

data[['previous_50_numbers', 'previous_50_colours']] = data.apply(previous_counts, axis=1, args=(data,))
    
data.head()

In [ ]:
data['region'] = data.apply(lambda x: (x['target_number'], x['target_colour']), axis=1)
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier  

In [ ]:
X = data[['count_of_the_day']]
# y = data['region']
y = data['target_colour']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
k = 50
knn = KNeighborsClassifier(n_neighbors=k)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
accuracy = knn.score(X_test, y_test)
print(f"Accuracy of the k-NN model: {accuracy:.2f}")

In [ ]:
knn.predict([[195]])